In [1]:
# !git clone https://github.com/andreFelipeHFuck/OTRS.git

In [2]:
import numpy as np
import cv2

In [3]:
img = cv2.imread("./OTRS/images/picture49.jpg")
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Qt: Session management error: None of the authentication protocols specified are supported


In [4]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow('Image', gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# Redimensionando a imagem para

size = (320, 240)
img_resize = cv2.resize(gray, size)
cv2.imshow('Image', img_resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# gray_esp32 = cv2.cvtColor(img_esp32, cv2.COLOR_BGR2GRAY)
# cv2.imshow(gray_esp32)

In [7]:
# Rotacionar a imagem

(h, w) = gray.shape[:2]

center = (w // 2, h // 2)

angle = 20

scale = 1.0

#  Matriz de rotação

M = cv2.getRotationMatrix2D(center, angle, scale)

rotated_image = cv2.warpAffine(gray, M, (w, h))

cv2.imshow('Image', rotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Algoritmo de processamento de imagem

Baseada no artigo: Dispositivo IoT para monitorar indicador de
pressão através de processamento de imagem, na seção 3.2 é descrito o algoritmo utilizado para processar as imagem para obter os resultados de pressão do manometro. São descritos 7 passos para tratar a imagem antes do cálculo do ponteiro.


## Passo 1:

Definir a região de interesse (ROI - "Region of interest"), no caso do artigo a região de interesse é o centro da foto. Criando uma imagem com a dimensão de 120x120 pixels.

In [8]:
x1, y1 = 107, 80   # 100, 100
x2, y2 = 207, 170  # 220, 220

crop_image = rotated_image[y1:y2, x1:x2]

cv2.imshow('Image', crop_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Passo 2:

Segmentação para retirar a cor da imagem, ou seja, transformar a imagem em tons de cinza. Esse passo foi feito já na abertura da imagem.**negrito**

## Passo 3:

Equalização do histograma, atenua o contraste da imagem, evidenciando a diferença entre as partes claras e **escuras**

## Passo 4:

Thresholding é processo de transformar a imagem em uma imagem binária, fazendo com que o pixel seja completamente branco ou completamente preto.

Para que as letras e outras marcações não estejam presentes na imagem binária é preciso utilizar um filtro linear na imagem antes de realizar um thresholding

In [9]:
# # Filtro de média

# blurred_image = cv2.blur(crop_image, (5, 5))
# cv2.imshow('Image', blurred_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [10]:
# equalise_image = cv2.equalizeHist(blurred_image)
# cv2.imshow('Image', equalise_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [11]:
# _, thresh_image = cv2.threshold(blurred_image, 99, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# cv2.imshow('Image', thresh_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [12]:
# Filtro gaussiano

gaussian_blurred_image = cv2.GaussianBlur(crop_image, (5, 5), 0)
cv2.imshow('Image', gaussian_blurred_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
_, gaussian_thresh_image = cv2.threshold(gaussian_blurred_image, 107, 255, cv2.THRESH_BINARY)

cv2.imshow('Image', gaussian_thresh_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
# # Filtro de mediano

# median_blurred_image = cv2.medianBlur(crop_image, 5)

# cv2.imshow('Image', median_blurred_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [15]:
# _, median_thresh_image = cv2.threshold(median_blurred_image, 99, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# cv2.imshow('Image', median_thresh_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

## Passo 7:

Cria um retângulo de área mínima que contempla toda a área preta do ponteiro da *imagem*

In [16]:
# Encontra o retângulo de área mínima

black_pixels = np.argwhere(gaussian_thresh_image == 0)

if black_pixels.size > 0:
  min_y, min_x = np.min(black_pixels, axis=0)
  max_y, max_x = np.max(black_pixels, axis=0)

  min_rect = cv2.minAreaRect(np.array([(min_x, min_y), (max_x, max_y)]))

center, _, _ = min_rect

(c_x, c_y) = center

(int(c_x), int(c_y))

(49, 47)

In [17]:
x1, y1 = 0, 0
x2, y2 = 49, 47

Q1_crop_image = gaussian_thresh_image[y1:y2, x1:x2]

cv2.imshow('Image', Q1_crop_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
# Porcentagem de pixels preto na tela

white_pixel_count = cv2.countNonZero(Q1_crop_image)
total_pixels = Q1_crop_image.size

black_pixels_count = total_pixels - white_pixel_count

porcent_black_pixels = black_pixels_count / total_pixels

int(porcent_black_pixels * 100)

0

In [19]:
x1, y1 = 49, 0
x2, y2 = 99, 47

Q2_crop_image = gaussian_thresh_image[y1:y2, x1:x2]

cv2.imshow('Image', Q2_crop_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
# Porcentagem de pixels preto na tela

white_pixel_count = cv2.countNonZero(Q2_crop_image)
total_pixels = Q2_crop_image.size

black_pixels_count = total_pixels - white_pixel_count

porcent_black_pixels = black_pixels_count / total_pixels

int(porcent_black_pixels * 100)

41

In [21]:
x1, y1 = 0, 47   #
x2, y2 = 49, 87

Q3_crop_image = gaussian_thresh_image[y1:y2, x1:x2]

cv2.imshow('Image', Q3_crop_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
# Porcentagem de pixels preto na tela

white_pixel_count = cv2.countNonZero(Q3_crop_image)
total_pixels = Q3_crop_image.size

black_pixels_count = total_pixels - white_pixel_count

Q3_porcent_black_pixels = black_pixels_count / total_pixels

Q3_porcent_black_pixels = int(Q3_porcent_black_pixels * 100)

Q3_porcent_black_pixels

15

In [23]:
x1, y1 = 49, 47
x2, y2 = 99, 87

Q4_crop_image = gaussian_thresh_image[y1:y2, x1:x2]

cv2.imshow('Image', Q4_crop_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# Porcentagem de pixels preto na tela

white_pixel_count = cv2.countNonZero(Q4_crop_image)
total_pixels = Q4_crop_image.size

black_pixels_count = total_pixels - white_pixel_count

Q4_porcent_black_pixels = black_pixels_count / total_pixels

Q4_porcent_black_pixels = int(Q4_porcent_black_pixels * 100)

Q4_porcent_black_pixels

11

In [25]:
Q3_porcent_black_pixels > Q4_porcent_black_pixels

True